In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
PRIMME INCLINATION CALCULATION - HIPERGATOR JUPYTER NOTEBOOK
==========================================================

Interactive Jupyter notebook for calculating grain boundary inclination vectors
from phase field simulation data on HiPerGator supercomputing cluster. This
notebook provides an interactive environment for PRIMME validation studies with
real-time visualization and analysis capabilities.

PRIMME Validation Framework:
- Parallel Runge-Kutta-based Implicit Multi-physics Multi-scale Engine
- Interactive comparison: Phase Field vs SPPARKS vs PRIMME methods  
- Real-time grain boundary inclination distribution analysis
- Visual validation of algorithm convergence and accuracy

Notebook Features:
- Step-by-step execution with intermediate results
- Interactive parameter adjustment and testing
- Real-time performance monitoring and visualization
- Flexible analysis of individual simulation cases
- Development and debugging environment for algorithm testing

Target Platform: UF HiPerGator Jupyter Hub
- Environment: Python 3.9+ with scientific computing stack
- Storage: Blue file system access (/blue/michael.tonks/)
- Processing: Multi-core parallel execution within notebook kernel
- Visualization: Matplotlib for real-time plotting

Scientific Context:
This notebook supports research into grain boundary character distribution
accuracy by providing an interactive environment for processing simulation
results and enabling real-time comparative analysis of inclination predictions.

Usage Workflow:
1. Load phase field simulation data from HDF5 files
2. Configure bilinear smoothing algorithm parameters
3. Execute inclination calculation with progress monitoring
4. Visualize results and validate algorithm performance
5. Export processed data for comparative analysis

Author: lin.yang
Created: Thu Sep 30 14:55:28 2021
Purpose: Interactive PRIMME validation and algorithm development
"""

# =================================================================
# ENVIRONMENT SETUP AND PATH CONFIGURATION
# =================================================================

import os
current_path = os.getcwd()  # Get current working directory
import sys

# Add project paths for module imports
sys.path.append(current_path)        # Current directory
sys.path.append('./../../.')         # VECTOR framework root directory

# =================================================================
# SCIENTIFIC COMPUTING AND DATA PROCESSING IMPORTS
# =================================================================

import numpy as np                    # Numerical arrays and mathematical operations
import math                          # Mathematical functions
from itertools import repeat        # Iterator utilities
import h5py                          # HDF5 file format for large datasets
import matplotlib.pyplot as plt     # Plotting and visualization
from tqdm import tqdm               # Progress bar for loop monitoring

# =================================================================
# VECTOR FRAMEWORK MODULES
# =================================================================

import myInput                              # Input parameter configuration
import PACKAGE_MP_Linear as smooth          # 2D Bilinear smoothing algorithm

print("Environment initialized successfully!")
print(f"Working directory: {current_path}")
print("All modules imported and ready for PRIMME validation analysis.")

In [ ]:
if __name__ == '__main__':
    """
    MAIN EXECUTION: INTERACTIVE PRIMME VALIDATION ANALYSIS
    ====================================================
    
    This interactive cell demonstrates the complete workflow for processing
    a single phase field simulation case and calculating inclination vectors
    for PRIMME validation. The example uses Case2AS_T3 which represents a
    standard grain boundary evolution scenario.
    
    Case Study: Case2AS_T3_tstep_300_600
    - Scenario: Case 2A (standard parameters) 
    - Variant: Standard evolution (S)
    - Trial: 3 (statistical replication)
    - Timesteps: 300 (early) to 600 (intermediate evolution)
    """

    # =================================================================
    # FILE PATHS AND DATA SPECIFICATION
    # =================================================================
    
    # HiPerGator Blue file system paths for demonstration case
    input_name = "/blue/michael.tonks/share/PRIMME_Inclination/Case2AS_T3_tstep_300_600.h5"
    output_name = "/blue/michael.tonks/share/PRIMME_Inclination_npy_files/Case2AS_T3_tstep_300_600_inclination_"
    
    print("="*60)
    print("PRIMME VALIDATION: INTERACTIVE INCLINATION ANALYSIS")
    print("="*60)
    print(f"Input file:  {input_name}")
    print(f"Output prefix: {output_name}")
    
    # =================================================================
    # HDF5 DATA LOADING AND EXTRACTION
    # =================================================================
    
    print("\nLoading phase field simulation data from HDF5...")
    
    # Open HDF5 file containing phase field simulation results
    f = h5py.File(input_name, 'r')
    
    # Extract all datasets from HDF5 file structure and create global variables
    # This approach preserves original variable names from simulation output
    print("Extracting datasets from HDF5 structure:")
    
    for simu in f.keys():
        print(f"  Processing simulation group: {simu}")
        DataContainer = f.get(simu)
        
        for dataset in DataContainer.keys():
            tmpdata = DataContainer[dataset]
            # Clean dataset names for Python variable compatibility
            dataset_clean = dataset.replace(' ','_')
            # Create global variables for algorithm compatibility
            globals()[dataset_clean] = tmpdata
            print(f"    → {dataset} → {dataset_clean}")
    
    f.close()  # Close HDF5 file to free memory
    print("✓ HDF5 data successfully loaded and extracted")
    
    # =================================================================
    # MICROSTRUCTURE PARAMETERS AND ANALYSIS SETUP
    # =================================================================
    
    # Extract key simulation parameters from loaded data
    steps, nz, nx, ny = ims_id.shape      # ims_id: microstructure ID array
    ng = len(euler_angles)                # euler_angles: grain crystallographic orientations
    
    print(f"\nMicrostructure Analysis Parameters:")
    print(f"  Domain dimensions: {nx} × {ny} lattice sites")
    print(f"  Z-layers: {nz} (typically 1 for 2D)")
    print(f"  Available timesteps: {steps}")
    print(f"  Total grain population: {ng}")
    print(f"  Microstructure data shape: {ims_id.shape}")
    
    # Display timestep range and resolution
    if hasattr(globals().get('time_steps', None), '__len__'):
        print(f"  Timestep range: {min(time_steps)} to {max(time_steps)}")
    
    print("\n✓ Parameters extracted and validated for inclination analysis")

In [ ]:
    # =================================================================
    # MAIN PROCESSING MATRIX INITIALIZATION
    # =================================================================
    
    # Initialize storage matrix for all processed inclination data
    # Shape: (timesteps, 3, nx, ny) where dimension 1 contains:
    # [0,:,:] = microstructure ID array
    # [1,:,:] = normalized x-component of inclination vectors  
    # [2,:,:] = normalized y-component of inclination vectors
    main_matrix = np.zeros((steps, 3, nx, ny))
    
    print(f"\nInitialized main storage matrix: {main_matrix.shape}")
    print("Matrix organization: [timestep, component, x_coord, y_coord]")
    
    # =================================================================
    # TIMESTEP PROCESSING LOOP WITH INCLINATION CALCULATION
    # =================================================================
    
    # Process specific timesteps for demonstration (early and intermediate states)
    target_timesteps = [300, 1600]  # Can be modified for different analysis needs
    
    print(f"\nProcessing inclination vectors for timesteps: {target_timesteps}")
    print("="*50)
    
    for i in tqdm(target_timesteps, desc="Calculating inclinations"):
        print(f"\n--- TIMESTEP {i} PROCESSING ---")
        
        # =================================================================
        # MICROSTRUCTURE EXTRACTION AND PREPARATION
        # =================================================================
        
        # Extract microstructure at current timestep
        microstructure = ims_id[i, :]           # Get 3D slice from data
        microstructure = np.squeeze(microstructure)  # Remove singleton dimensions
        
        print(f"  Microstructure shape: {microstructure.shape}")
        print(f"  Grain ID range: {np.min(microstructure)} to {np.max(microstructure)}")
        print(f"  Active grains: {len(np.unique(microstructure))}")
        
        # Initialize gradient field for bilinear smoothing algorithm
        R = np.zeros((nx, ny, 2))  # [x-gradient, y-gradient] components
        
        # =================================================================
        # BILINEAR SMOOTHING ALGORITHM EXECUTION
        # =================================================================
        
        print("  Executing bilinear smoothing algorithm...")
        
        # Algorithm configuration for optimal performance
        cores = 8          # Parallel processing cores (adjust for your system)
        loop_times = 5     # Maximum smoothing iterations for convergence
        
        # Initialize bilinear smoothing class
        # Parameters: (nx, ny, ng, cores, max_iterations, microstructure, gradient_field, param1, debug_mode)
        test1 = smooth.linear_class(nx, ny, ng, cores, loop_times, microstructure, R, 0, False)
        
        # Execute inclination calculation using bilinear smoothing
        test1.linear_main('inclination')
        
        # Extract calculated inclination field
        P = test1.get_P()  # Returns [microstructure, x_component, y_component]
        
        print(f"  ✓ Smoothing completed successfully")
        print(f"  Calculated field shape: {np.array(P).shape}")
        
        # =================================================================
        # PERFORMANCE MONITORING (OPTIONAL REPORTING)
        # =================================================================
        
        # Uncomment for detailed performance analysis
        print(f"  Algorithm performance:")
        print(f"    Actual iterations: {test1.loop_times}")
        print(f"    Total runtime: {test1.running_time:.2f} seconds")
        print(f"    Core processing time: {test1.running_coreTime:.2f} seconds")
        print(f"    Parallel efficiency: {(test1.running_coreTime/test1.running_time)*100:.1f}%")

        # =================================================================
        # INCLINATION VECTOR PROCESSING AND NORMALIZATION
        # =================================================================
        
        print("  Processing and normalizing inclination vectors...")
        
        # Convert to numpy array for mathematical operations
        P_final = np.array(P)
        
        # Standard coordinate transformation for inclination representation
        # This follows crystallographic convention for grain boundary inclination
        P_final[1] = -P[2]  # x-component = -original_y_component
        P_final[2] = P[1]   # y-component = original_x_component
        
        # Normalize inclination vectors to unit length
        # Calculate magnitude: sqrt(x² + y²)
        magnitude = (P_final[1]**2 + P_final[2]**2)**0.5
        
        # Avoid division by zero: only normalize non-zero vectors
        mask = magnitude != 0
        P_final[1][mask] = P_final[1][mask] / magnitude[mask]
        P_final[2][mask] = P_final[2][mask] / magnitude[mask]
        
        # Handle any remaining NaN values (replace with zero)
        P_final = np.nan_to_num(P_final)
        
        # Validation of normalized vectors
        print(f"  Vector normalization complete:")
        print(f"    X-component range: [{np.min(P_final[1]):.3f}, {np.max(P_final[1]):.3f}]")
        print(f"    Y-component range: [{np.min(P_final[2]):.3f}, {np.max(P_final[2]):.3f}]")
        print(f"    Non-zero vectors: {np.sum(magnitude > 0)} / {magnitude.size}")
        
        # Store processed data in main matrix
        main_matrix[i] = P_final
        
        # =================================================================
        # DATA OUTPUT AND PERSISTENCE
        # =================================================================
        
        # Save processed inclination data for current timestep
        output_filename = output_name + f"step{i}"
        np.save(output_filename, P_final)
        
        print(f"  ✓ Saved inclination data: {output_filename}.npy")
        print(f"  File size: {P_final.nbytes / (1024**2):.1f} MB")
        
    # =================================================================
    # COMPLETION SUMMARY AND DATA ORGANIZATION
    # =================================================================
    
    print("\n" + "="*60)
    print("INCLINATION CALCULATION COMPLETED SUCCESSFULLY!")
    print("="*60)
    
    print(f"Processed timesteps: {target_timesteps}")
    print(f"Generated files: {len(target_timesteps)}")
    print(f"Output directory: {os.path.dirname(output_name)}")
    
    print(f"\nData Matrix Organization:")
    print(f"  main_matrix[timestep, component, x, y]")
    print(f"  • timestep: Index of simulation time")
    print(f"  • component: [0=microstructure, 1=x_vector, 2=y_vector]")
    print(f"  • x, y: Spatial coordinates in microstructure domain")
    
    print(f"\nData Specifications:")
    print(f"  • Inclination vectors: Unit-normalized (magnitude = 1)")
    print(f"  • Coordinate system: Standard crystallographic convention")
    print(f"  • NaN handling: Replaced with zeros for stability")
    print(f"  • File format: NumPy binary (.npy) for fast loading")
    print(f"  • Compatibility: Ready for comparative analysis scripts")
    
    print(f"\nNext Steps for Analysis:")
    print("1. Load data in comparative analysis notebooks")
    print("2. Calculate grain boundary character distributions")  
    print("3. Generate polar plots for method validation")
    print("4. Perform statistical accuracy assessment")
    print("5. Compare with SPPARKS and other simulation methods")
    
    # Data organization comment for reference
    # main_matrix[time_step, inclination_axis, x, y] stores all calculated inclination data
    # • First index (time_step): Temporal evolution stage index
    # • Second index (inclination_axis): Physical component (0=microstructure, 1=x-vector, 2=y-vector)  
    # • Third index (x): Spatial x-coordinate in microstructure
    # • Fourth index (y): Spatial y-coordinate in microstructure
    
    print(f"\n✓ Interactive PRIMME validation analysis complete!")